In [3]:
from Model.model import ModelSTGCN
import torch
from torch.utils.data import DataLoader,Dataset
import pandas as pds
import numpy as np

In [20]:
class ActionDataset(Dataset):
    def __init__(self,Data):
        super().__init__()
        self.N_frame=Data['total_frames']
        kpscore=np.expand_dims(Data['kp_score'][0],axis=2)
        kp=Data['kp'][0]
        self.data=np.concatenate((kp,kpscore),axis=2)

    def __getitem__(self, index):
        return self.data[index]

    def __len__(self):
        return self.N_frame   

In [12]:
kp=datalst[0]['kp'][0]
kp_score=datalst[0]['kp_score'][0]
kp_score=np.expand_dims(kp_score,axis=2)
a=np.concatenate((kp,kp_score),axis=2)
a[5]


In [26]:
datalst=pds.read_pickle('train.pkl')
actDataset=ActionDataset(datalst[0])
dataloader=DataLoader(dataset=actDataset,batch_size=4,shuffle=True)
dataiter=iter(dataloader)
data=dataiter.next()
data.shape

torch.Size([4, 17, 3])

In [24]:
data

array([[0.70539933, 0.29457283, 0.8300471 ],
       [0.70950854, 0.287279  , 0.8444217 ],
       [0.70539933, 0.283632  , 0.8189148 ],
       [0.70950854, 0.287279  , 0.72429085],
       [0.69307166, 0.27269128, 0.8215558 ],
       [0.6992355 , 0.31280747, 0.7642889 ],
       [0.66636175, 0.29821974, 0.8369639 ],
       [0.71156317, 0.36751142, 0.7903819 ],
       [0.6622526 , 0.38209915, 0.73215187],
       [0.7300547 , 0.41492158, 0.58510613],
       [0.69307166, 0.34198296, 0.80920196],
       [0.67047095, 0.41492158, 0.70603204],
       [0.64992493, 0.40033385, 0.7552263 ],
       [0.66636175, 0.4842132 , 0.80608684],
       [0.63554263, 0.47691935, 0.72092974],
       [0.6396519 , 0.56079876, 0.8324282 ],
       [0.6129419 , 0.5535049 , 0.74012715]], dtype=float32)